In [1]:
import sys
import os
import numpy as np
import pandas as pd
from collections import Counter
from operator import itemgetter

In [2]:
def rawdata(map_file):
    lis_dicts=[]
    pDict = {}
    mDict = {}
    infile = open(map_file, 'r')
    line = infile.readline()
    while line != '':
        fields = line.split()
        col1 = str(fields[0])   #strand; note: if sequencing was performed without barcode reading, the column numbering is changed
        col2 = int(fields[1])   #left-most position
        col3 = str(fields[2])   #footprint seq
        length = len(col3)      #footprint length        
        if col1 == '+': #for plus strand
            pend5 = col2 + 1                #Bowtie uses 0-based offset: transform to 1-based and subtract 1st base 
            if pend5 in pDict:
                pDict[pend5] += 1.0
            else:
                pDict[pend5] = 1.0 
        elif col1 == '-':               #for minus strand
            end3 = col2 + 1         #for minus strand, Bowtie gives leftmost position (3' end) with zero-based numbering
            mend5 = end3 + length - 1
            if mend5 in mDict:
                mDict[mend5] += 1.0
            else:
                mDict[mend5] = 1.0 
        else:
                pass
        line = infile.readline()
    for pend5 in range(1,4641653):          
        if pend5 not in pDict:
            pDict[pend5] = 0
    p_list=[(p, pDict[p]) for p in sorted(pDict)]
    for mend5 in range(1,4641653):          
        if mend5 not in mDict:
            mDict[mend5] = 0
    m_list=[(m, mDict[m]) for m in sorted(mDict)]
    infile.close()
    lis_dicts.append(p_list)
    lis_dicts.append(m_list)
    return lis_dicts

def get_local_max_signal(lis_dicts):
    lis_pause_lis=[]
    for lis in lis_dicts:
        numeric_list=[(int(pos),float(count))for pos,count in lis]
        trunc_list=numeric_list[0:100000]
        pause_list=[]
        win_side=25
        for i in range(win_side,100000-win_side): #range from list_m[0](which equals pos 1 in genome) to list_m[4639651] (not including list_m[4639651] by default)
            window=trunc_list[(i-win_side):(i+win_side+1)] #1st window: list_m[0]-list_m[51](not including list[51] by default)
            dic_freq=Counter(mem[1] for mem in window) #count the number oftimes each signal appear in window. goal: filter regions with multiple equal local max signal
            win_count_lis=[count for pos,count in window]
            win_count_mean=np.array(win_count_lis).mean()
            win_count_std=np.array(win_count_lis).std()
            win_count_max=max(window,key=itemgetter(1))[1]
            if window[win_side][1]==win_count_max and dic_freq[window[win_side][1]]==1 and win_count_max>=(win_count_mean + win_count_std*5):  #identify window max for signal of reads.
                pause_list.append(window[win_side])
            else:
                pass
        lis_pause_lis.append(pause_list)
    return lis_pause_lis


In [3]:
map_file=os.path.join('netseq_tutorial/wt_mmc_NET.map')
lis_dicts=rawdata(map_file)

In [5]:
lis_pause_lis=get_local_max_signal(lis_dicts)
print (lis_pause_lis[1][0:100])

[(67, 2.0), (258, 1517.0), (297, 424.0), (409, 105.0), (628, 22.0), (678, 19.0), (813, 23.0), (942, 17.0), (1041, 24.0), (1092, 18.0), (1214, 19.0), (1437, 10.0), (1523, 19.0), (1602, 14.0), (1689, 13.0), (2018, 12.0), (2636, 36.0), (2678, 21.0), (2726, 99.0), (3114, 18.0), (3434, 26.0), (3525, 30.0), (3597, 28.0), (3625, 10.0), (3659, 18.0), (3772, 9.0), (3801, 27.0), (3960, 26.0), (4043, 10.0), (4160, 20.0), (4262, 3.0), (4483, 23.0), (4725, 9.0), (4810, 13.0), (4949, 14.0), (5209, 14.0), (5359, 9.0), (5446, 12.0), (5617, 1.0), (5713, 8.0), (5901, 11.0), (5995, 5.0), (6127, 2.0), (6270, 2.0), (6302, 4.0), (6470, 3.0), (6555, 3.0), (6613, 3.0), (6684, 3.0), (6761, 5.0), (6820, 5.0), (6935, 2.0), (6964, 2.0), (7076, 4.0), (7239, 4.0), (7480, 1.0), (7537, 1.0), (7599, 2.0), (7653, 4.0), (7912, 4.0), (8006, 1.0), (8129, 2.0), (8302, 86.0), (8433, 251.0), (8585, 54.0), (8628, 43.0), (8660, 61.0), (8822, 28.0), (8911, 71.0), (8954, 74.0), (9080, 143.0), (9148, 51.0), (9229, 150.0), (9383, 